In [1]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.5 MB/s eta 0:00:00


In [2]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=6692a7bf727361619982870c18a86258407c08f60866c3dcf2544568e8b76957
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [3]:
# from getpass import getpass
# HF_TOKEN = getpass("Paste your HF token: ")


In [ ]:
# from huggingface_hub import login
# login(token=HF_TOKEN)   


In [5]:
# import os
# os.environ["HF_TOKEN"] = HF_TOKEN


In [6]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import re
import json
from pathlib import Path
import math
import time
import argparse
import random
from dataclasses import dataclass, asdict
from typing import List, Dict, Tuple, Optional
import argparse

# few-shot helpers
from fewshot import load_shots, with_few_shot, make_shots_from_silver


import numpy as np
import pandas as pd

# HF datasets + transformers
from datasets import load_dataset
from evaluate import load as load_metric
from rouge_score import rouge_scorer

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# =========================
# Global Config & Utilities
# =========================

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

# ROUGE (evaluate) for greedy/beam/local/global scoring
_evaluate_rouge = load_metric("rouge")

def rouge_score(candidate: str, reference: str, rougex: str = "rougeL") -> float:
    """
    Return ROUGE F1 (rouge1|rouge2|rougeL).
    """
    res = _evaluate_rouge.compute(
        predictions=[candidate],
        references=[reference],
        rouge_types=[rougex]
    )
    return float(res[rougex])

def rouge_multi(candidate: str, reference: str) -> Dict[str, float]:
    """
    Compute ROUGE-1/2/L F1 using 'evaluate'.
    """
    res = _evaluate_rouge.compute(
        predictions=[candidate],
        references=[reference],
        rouge_types=["rouge1", "rouge2", "rougeL"]
    )
    return {k: float(v) for k, v in res.items()}

def normalize_text(s: str) -> str:
    s = s.lower()
    s = s.replace("\u2019", "'").replace("\u201c", '"').replace("\u201d", '"').replace("\u2014","-")
    s = re.sub(r"\s+", " ", s.strip())
    return s

# ===========================
# Sentence Splitting
# ===========================

def _try_nltk_sent_tokenize(text: str) -> List[str]:
    try:
        import nltk
        try:
            nltk.data.find('tokenizers/punkt')
        except LookupError:
            nltk.download('punkt', quiet=True)
        from nltk.tokenize import sent_tokenize
        return [s.strip() for s in sent_tokenize(text) if s.strip()]
    except Exception:
        return []

def _regex_sentence_split(text: str) -> List[str]:
    # Simple fallback splitter
    chunks = re.split(r'(?<=[.!?])\s+(?=[A-Z0-9])', text.strip())
    out = []
    for c in chunks:
        c = c.strip()
        if c:
            out.append(c)
    return out

def split_sentences(text: str) -> List[str]:
    sents = _try_nltk_sent_tokenize(text)
    if sents:
        return sents
    return _regex_sentence_split(text)

# =======================================
# Heuristics from Track B (with tiny tweaks)
# =======================================

def greedy_extractive_summary(sentences: List[str],
                              abstractive_summary: str,
                              eval_criteria: str,
                              max_sent: int) -> List[str]:
    selected = []
    remaining = sentences[:]
    while remaining and max_sent > 0:
        best_sentence = None
        best_score = -1.0
        for sent in remaining:
            cand = " ".join(selected + [sent])
            r = rouge_score(cand, abstractive_summary, eval_criteria)
            if r > best_score:
                best_score = r
                best_sentence = sent
        selected.append(best_sentence)
        remaining.remove(best_sentence)
        max_sent -= 1
    return selected

def beam_extractive_summary(sentences: List[str],
                            abstractive_summary: str,
                            eval_criteria: str,
                            max_sent: int,
                            beam_size: int = 4,
                            with_score: bool = False):
    beams = [([], 0.0)]
    for _ in range(min(max_sent, len(sentences))):
        new_beams = []
        for selected, _ in beams:
            remaining = [s for s in sentences if s not in selected]
            for sent in remaining:
                cand = " ".join(selected + [sent])
                r = rouge_score(cand, abstractive_summary, eval_criteria)
                new_beams.append((selected + [sent], r))
        if not new_beams:
            break
        new_beams.sort(key=lambda x: x[1], reverse=True)
        beams = new_beams[:beam_size]
    if with_score:
        return beams
    return beams[0][0] if beams else []

def local_extractive_summary(sentences: List[str],
                             abstractive_summary: str,
                             eval_criteria: str,
                             max_sent: int) -> List[str]:
    scored = []
    for s in sentences:
        r = rouge_score(s, abstractive_summary, eval_criteria)
        scored.append((s, r))
    scored.sort(key=lambda x: x[1], reverse=True)
    return [s for s, _ in scored[:max_sent]]

def global_extractive_summary(sentences: List[str],
                              abstractive_summary: str,
                              eval_criteria: str,
                              max_sent: int,
                              beam_size: int = 4) -> List[str]:
    candidates = beam_extractive_summary(sentences, abstractive_summary, eval_criteria,
                                         max_sent, beam_size, with_score=True)
    uniq = list(set(sent for cand, _ in candidates for sent in cand))
    sent2score = []
    for s in uniq:
        acc = 0.0
        for cand, score in candidates:
            if s in cand:
                acc += score
        sent2score.append((s, acc))
    sent2score.sort(key=lambda x: x[1], reverse=True)
    return [s for s, _ in sent2score[:max_sent]]

# =============================
# Track C:"BEST" heuristic
# =============================

@dataclass
class HeuristicConfig:
    name: str
    eval_criteria: str
    grid_k_min: int
    grid_k_max: int
    beam_size: int = 4

# Map Track B names to Track C names
_B2C_NAME = {
    "greedy_search": "greedy",
    "beam_search":   "beam",
    "local_score":   "local",
    "global_score":  "global",
    # also accept already-normalized names
    "greedy": "greedy",
    "beam":   "beam",
    "local":  "local",
    "global": "global",
}

def load_best_from_trackB(path="best_heuristic.json"):
    p = Path(path)
    if not p.exists():
        return None
    with open(p, "r") as f:
        cfg = json.load(f)

    name_raw = cfg.get("name", "global")
    name = _B2C_NAME.get(name_raw, "global")

    return {
        "name": name,
        "eval_criteria": cfg.get("eval_criteria", "rougeL"),
        "grid_k_min": int(cfg.get("grid_k_min", 1)),
        "grid_k_max": int(cfg.get("grid_k_max", 32)),
        "beam_size":  int(cfg.get("beam_size", 4)),
        "best_k_single": int(cfg.get("best_k_single", 0)),
        "best_k_all_together": int(cfg.get("best_k_all_together", 0)),
        "source": cfg.get("source", "TrackB"),
    }

# allow a fixed-K shortcut (skip grid search to speed up)
USE_FIXED_K: int | None = None

# Load best heuristic from Track B results
_b = load_best_from_trackB("best_heuristic.json")
if _b:
    BEST_HEURISTIC = HeuristicConfig(
        name=_b["name"],
        eval_criteria=_b["eval_criteria"],
        grid_k_min=_b["grid_k_min"],
        grid_k_max=_b["grid_k_max"],
        beam_size=_b["beam_size"],
    )
    USE_FIXED_K = _b.get("best_k_single") or None
    #USE_FIXED_K = _b.get("best_k_all_together") or _b.get("best_k_single") or None
else:
    BEST_HEURISTIC = HeuristicConfig(
        name="global",
        eval_criteria="rougeL",
        grid_k_min=1,
        grid_k_max=32,
        beam_size=4
    )
# check
if USE_FIXED_K is not None:
    print(f"[Track C] Using FIXED K = {USE_FIXED_K} from best_heuristic.json (no grid search).")
else:
    print(f"[Track C] No fixed K provided -> running grid search K∈[{BEST_HEURISTIC.grid_k_min},{BEST_HEURISTIC.grid_k_max}].")

def run_heuristic(sentences: List[str], abstractive: str, cfg: HeuristicConfig, K: int) -> List[str]:
    if cfg.name == "greedy":
        return greedy_extractive_summary(sentences, abstractive, cfg.eval_criteria, K)
    elif cfg.name == "beam":
        return beam_extractive_summary(sentences, abstractive, cfg.eval_criteria, K, cfg.beam_size)
    elif cfg.name == "local":
        return local_extractive_summary(sentences, abstractive, cfg.eval_criteria, K)
    elif cfg.name == "global":
        return global_extractive_summary(sentences, abstractive, cfg.eval_criteria, K, cfg.beam_size)
    else:
        raise ValueError(f"Unknown heuristic: {cfg.name}")

def select_k_per_doc(sentences: List[str],
                     abstractive: str,
                     cfg: HeuristicConfig,
                     fixed_k: int | None = None) -> Tuple[List[str], int, Dict[str, float]]:
    """
    If fixed_k is provided, use that K directly (no grid search).
    Otherwise grid-search K in [cfg.grid_k_min, cfg.grid_k_max].
    """
    if fixed_k is not None:
        K = min(max(fixed_k, cfg.grid_k_min), max(cfg.grid_k_min, min(cfg.grid_k_max, len(sentences))))
        sel = run_heuristic(sentences, abstractive, cfg, K)
        cand = " ".join(sel)
        rs = rouge_multi(cand, abstractive)
        return sel, K, rs


    best_k, best_score, best_selection, best_rouge = 1, -1.0, [], {}
    k_max = min(cfg.grid_k_max, max(len(sentences), cfg.grid_k_min))
    for K in range(cfg.grid_k_min, k_max + 1):
        sel = run_heuristic(sentences, abstractive, cfg, K)
        cand = " ".join(sel)
        rs = rouge_multi(cand, abstractive)
        score = rs[cfg.eval_criteria]
        if score > best_score:
            best_score = score
            best_k = K
            best_selection = sel
            best_rouge = rs
    return best_selection, best_k, best_rouge


# ============================================
# Datasets: CNN/DailyMail and XSum (abstractive)
# ============================================

def _normalize_split(split: str) -> str:
    s = split.strip().lower()
    return {"val": "validation", "dev": "validation", "valid": "validation"}.get(s, s)

def _ensure_str_id(raw_id, fallback: str) -> str:
    try:
        return str(raw_id) if raw_id is not None else fallback
    except Exception:
        return fallback

def _load_xsum_parquet(split: str):
    """
    Load XSum from the Hub Parquet export to avoid script-based loaders
    """
    split = _normalize_split(split)
    if split not in {"train", "validation", "test"}:
        raise ValueError(f"Unsupported split: {split}")

    # Parquet branch + layout:
    # refs/convert/parquet/default/{train|validation|test}/*.parquet
    files = {
        "train": "hf://datasets/EdinburghNLP/xsum@refs/convert/parquet/default/train/*.parquet",
        "validation": "hf://datasets/EdinburghNLP/xsum@refs/convert/parquet/default/validation/*.parquet",
        "test": "hf://datasets/EdinburghNLP/xsum@refs/convert/parquet/default/test/*.parquet",
    }
    ds = load_dataset("parquet", data_files={split: files[split]}, split=split)
    return ds

def load_abstractive_dataset(name: str, split: str):
    """
    name: 'cnndm' or 'xsum'
    returns iterable of dicts: {id, document, reference}
    """
    n = name.lower()
    split_norm = _normalize_split(split)

    if n in ("cnndm", "cnn_dm", "cnn/dailymail", "cnn-dailymail"):
        ds = load_dataset("cnn_dailymail", "3.0.0", split=split_norm)
        return [{
            "id": _ensure_str_id(ex.get("id"), f"cnndm-{split_norm}-{i}"),
            "document": ex["article"],
            "reference": ex["highlights"],
        } for i, ex in enumerate(ds)]

    elif n == "xsum":
        ds = _load_xsum_parquet(split_norm)
        # Parquet columns are: id, document, summary
        # Map to thw schema
        data = []
        for i, ex in enumerate(ds):
            doc = ex.get("document", "")
            summ = ex.get("summary", "")
            if not doc or not summ:
                continue
            data.append({
                "id": _ensure_str_id(ex.get("id"), f"xsum-{split_norm}-{i}"),
                "document": doc,
                "reference": summ,
            })
        return data

    else:
        raise ValueError("Unsupported dataset. Use 'cnndm' or 'xsum'.")


# ===================================
# Silver dataset creation (Track C pt.1)
# ===================================

def sentences_to_indices(selected: List[str], source: List[str]) -> List[int]:
    # mapping with duplicates handling
    lut = {}
    for i, s in enumerate(source):
        ns = normalize_text(s)
        lut.setdefault(ns, []).append(i)
    used = set()
    out = []
    for s in selected:
        ns = normalize_text(s)
        cand = lut.get(ns, [])
        idx = next((i for i in cand if i not in used), cand[0] if cand else None)
        if idx is None:
            # fallback: substring
            idx = next((i for i, src in enumerate(source)
                        if ns in normalize_text(src) and i not in used), None)
        if idx is not None:
            out.append(idx)
            used.add(idx)
    return sorted(out)

def make_silver(df_name: str,
                split: str,
                out_path: str,
                max_docs: Optional[int],
                cfg: HeuristicConfig):
    """
    Build silver extractive labels for the given abstractive dataset and save to parquet.
    """
    docs = load_abstractive_dataset(df_name, split)
    if max_docs is not None:
        docs = docs[:max_docs]

    rows = []
    t0 = time.perf_counter()
    for j, ex in enumerate(docs):
        doc_id = ex["id"]
        doc = ex["document"]
        ref = ex["reference"]
        sents = split_sentences(doc)
        if len(sents) == 0:
            continue

        # Grid search K and pick best
        # selected_sents, best_k, rouge_dict = select_k_per_doc(sents, ref, cfg)
        selected_sents, best_k, rouge_dict = select_k_per_doc(
        sents, ref, cfg, fixed_k=USE_FIXED_K  # will skip grid if K from Track B is provided
        )
        selected_idx = sentences_to_indices(selected_sents, sents)

        rows.append({
            "id": doc_id,
            "sentences": sents,
            "abstractive_summary": ref,
            "selected_indices": selected_idx,  # 0-based indices of selected sentences
            "K_selected": best_k,
            "heuristic_name": cfg.name,
            "heuristic_params": asdict(cfg),
            "rouge1": rouge_dict.get("rouge1", None),
            "rouge2": rouge_dict.get("rouge2", None),
            "rougeL": rouge_dict.get("rougeL", None),
            "dataset_source": df_name,
            "split": split,
            "version": "C-0.1.0",
        })
        if (j+1) % 50 == 0:
            print(f"[{df_name}] processed {j+1}/{len(docs)}")

    dt = time.perf_counter() - t0
    print(f"Finished {df_name}:{split} -> {len(rows)} examples in {dt:.1f}s")

    os.makedirs(os.path.dirname(out_path), exist_ok=True)
    pd.DataFrame(rows).to_parquet(out_path, index=False)
    print(f"Saved silver dataset -> {out_path}")

# ==========================================
# Track A: Simple Prompt templates
# ==========================================

def simple_vanilla_prompt(sentences: List[str], max_k: int | None = None) -> str:
    cap = f"- Select AT MOST {max_k} sentences.\n" if max_k else ""
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    inp = "\n".join(numbered)
    return f"""You are an expert in extractive summarization. Your task is to select the most important sentences from the document.

Input:
{inp}

Rules:
- Select ONLY the most important sentences.
{cap}- If no sentences are important, return an empty list.
- Indices are 1-based.
- Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}"""


def least_to_most_simple_prompt(sentences, max_k=None):
    cap = f"- Select AT MOST {max_k} sentences.\n" if max_k else ""
    numbered = [f"Sentence {i+1}: {s}" for i,s in enumerate(sentences)]
    inp = "\n".join(numbered)
    return f"""You are an expert in extractive summarization. Your task is to select the most important sentences from the document.

Input:
{inp}

First, identify the main topics and key arguments of the document.
Then, select ONLY the sentences that directly relate to those topics.
{cap}- If no sentences are important, return an empty list.
- Indices are 1-based.
- Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}"""



def tool_augmented_simple_prompt(sentences: List[str]) -> str:
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    inp = "\n".join(numbered)
    return f"""You are an expert in extractive summarization. Your task is to select the most important sentences from the document.

Input:
{inp}

Instructions:
1. For each sentence, use the internal `check_importance(sentence)` tool.
2. The tool's output is 'important' if the sentence is central to the main idea, otherwise it is 'not_important'.
3. List the sentences that result in an 'important' output.
4. If no sentences are important, return an empty list.
5. Indices are 1-based.
6. Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}"""

def scoring_based_simple_prompt(sentences: List[str]) -> str:
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    inp = "\n".join(numbered)
    return f"""You are an expert in extractive summarization. Your task is to select the most important sentences from the document.

Input:
{inp}

Instructions:
1. For each sentence, assign a score from 1 (low importance) to 5 (high importance) for how central it is to the document's main idea.
2. Only select sentences with a score of 4 or 5.
3. If no sentences meet the threshold, return an empty list.
4. Indices are 1-based.
5. Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}"""

def self_ask_simple_prompt(sentences: List[str]) -> str:
    numbered = [f"Sentence {i+1}: {s}" for i, s in enumerate(sentences)]
    inp = "\n".join(numbered)
    return f"""You are an expert in extractive summarization. Your task is to select the most important sentences from the document.

Input:
{inp}

Instructions:
1. Reason step-by-step. First, ask the question: "What is the main idea of this document?"
2. Then, for each sentence, ask: "Does this sentence support the main idea?"
3. Compile a list of all sentences for which the answer was "Yes".
4. If no sentences are important, return an empty list.
5. Indices are 1-based.
6. Return ONLY valid JSON.

Return format:
{{"selected_sentences": [list_of_sentence_numbers]}}"""

PROMPT_STRATEGIES = {
    "vanilla": simple_vanilla_prompt,
    "ltm": least_to_most_simple_prompt,
    "tool": tool_augmented_simple_prompt,
    "scoring": scoring_based_simple_prompt,
    "selfask": self_ask_simple_prompt,
}

# ========================================
# Track A: Model loading + predict
# ========================================

GEN_KW = dict(
    max_new_tokens=192,
    do_sample=False,
    temperature=0.0,
    top_p=1.0,
    return_dict_in_generate=True
)

def load_model_and_tokenizer(model_name: str):
    import os, torch
    from transformers import AutoTokenizer, AutoModelForCausalLM

    token = os.getenv("HF_TOKEN", None)

    def is_qwen(name: str) -> bool:
        n = name.lower()
        return "qwen" in n

    def is_llama(name: str) -> bool:
        n = name.lower()
        return ("meta-llama" in n) or ("llama" in n)

    # choose dtype & device map
    use_cuda = torch.cuda.is_available()
    dtype = torch.bfloat16 if (use_cuda and torch.cuda.is_bf16_supported()) else torch.float16
    device_map = "auto" if use_cuda else None

    trust = True if is_qwen(model_name) else False

    try:
        tok = AutoTokenizer.from_pretrained(
            model_name,
            use_fast=True,
            trust_remote_code=trust,
            token=token,
        )
        mdl = AutoModelForCausalLM.from_pretrained(
            model_name,
            trust_remote_code=trust,
            token=token,
            torch_dtype=dtype,
            device_map=device_map,
        )
        if tok.pad_token_id is None:
            tok.pad_token = tok.eos_token
        mdl.eval()
        return mdl, tok
    except Exception as e:
        # diagnostics for gated Llama or typos
        print(f"[warn] Could not load '{model_name}': {e}")
        print("[hint] If this is a gated/private repo, make sure:")
        print("       1) You accepted the license on the model page")
        print("       2) You set HF_TOKEN in the environment and called huggingface_hub.login(token=HF_TOKEN)")
        print("       3) The repo id is exact")
        # public fallback to a model that works, so the notebook doesn’t die
        fallback = "Qwen/Qwen2.5-1.5B-Instruct"
        print(f"[info] Falling back to: {fallback}")
        tok = AutoTokenizer.from_pretrained(fallback, trust_remote_code=True, token=token)
        mdl = AutoModelForCausalLM.from_pretrained(
            fallback, trust_remote_code=True, token=token,
            torch_dtype=dtype, device_map=device_map
        )
        if tok.pad_token_id is None:
            tok.pad_token = tok.eos_token
        mdl.eval()
        return mdl, tok


def safe_extract_json_strict(text: str) -> Dict:
    text = text.strip()
    try:
        js = json.loads(text)
        if isinstance(js, dict) and "selected_sentences" in js:
            return js
        if isinstance(js, list):
            return {"selected_sentences": js}
    except Exception:
        pass
    text_clean = re.sub(r"^```[\w-]*\s*\n", "", text, flags=re.S)
    text_clean = re.sub(r"\n```$", "", text_clean, flags=re.S).strip()
    objs = re.findall(r"\{[\s\S]*?\}", text_clean)
    for s in reversed(objs):
        try:
            js = json.loads(s)
            if isinstance(js, dict) and "selected_sentences" in js:
                return js
        except Exception:
            continue
    m = re.search(r"(?m)^\s*\[(?:\s*\d+\s*(?:,\s*\d+\s*)*)?\]\s*$", text_clean)
    if m:
        try:
            arr = json.loads(m.group(0))
            return {"selected_sentences": arr}
        except Exception:
            pass
    return {}

def predict_indices_for_doc(model, tokenizer, prompt_builder, sentences: List[str],
                            system_msg: Optional[str] = "You are an expert in extractive summarization.") -> List[int]:
    #user_prompt = prompt_builder(sentences)
    user_prompt = prompt_builder(sentences, max_k=USE_FIXED_K)
    messages = [
        {"role": "system", "content": system_msg},
        {"role": "user", "content": user_prompt}
    ]
    chat_text = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    inputs = tokenizer(chat_text, return_tensors='pt', padding=True)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    gen = model.generate(**inputs, eos_token_id=tokenizer.eos_token_id,
                         pad_token_id=tokenizer.pad_token_id, **GEN_KW)
    new_ids = gen.sequences[:, inputs["input_ids"].shape[1]:]
    out = tokenizer.decode(new_ids[0], skip_special_tokens=True)
    js = safe_extract_json_strict(out)
    idxs = js.get("selected_sentences", [])
    n = len(sentences)
    cleaned = []
    for v in idxs:
        if isinstance(v, (int, float)):
            v = int(v)
            if 1 <= v <= n:
                cleaned.append(v)

        cleaned = sorted(set(cleaned))

    if USE_FIXED_K is not None and len(cleaned) > USE_FIXED_K:
        cleaned = cleaned[:USE_FIXED_K]  # or sorted(cleaned)[:USE_FIXED_K]

    return cleaned


    # return sorted(set(cleaned))

# ===============================
# Metrics for Benchmarking (Track A)
# ===============================

def prf1(pred: set, gold: set) -> Tuple[float, float, float]:
    tp = len(pred & gold)
    p = tp / max(len(pred), 1)
    r = tp / max(len(gold), 1)
    f1 = 2 * p * r / max(p + r, 1e-12)
    return p, r, f1

def indices_to_text(sentences: List[str], idxs_1b: List[int]) -> str:
    return " ".join(sentences[i-1] for i in idxs_1b if 1 <= i <= len(sentences))

# ============================
# Benchmark over Silver (C pt.2)
# ============================

def load_silver(path: str) -> pd.DataFrame:
    return pd.read_parquet(path)

def run_benchmark(silver_path: str,
                  model_name: str,
                  strategy: str,
                  max_docs: Optional[int] = None,
                  shots_path: Optional[str] = None,
                  shots_n: int = 0,
                  shots_seed: int = SEED):
    df = load_silver(silver_path)
    if max_docs is not None:
        df = df.iloc[:max_docs].copy()

    # Base (zero-shot) prompt builder
    prompt_builder_base = PROMPT_STRATEGIES[strategy]
    prompt_builder = prompt_builder_base

    # Few-shot wrapper (only if both flags provided)
    if shots_path and shots_n > 0:
        print(f"[Benchmark] few-shot ON | n={shots_n} seed={shots_seed} path={shots_path}")
        shots = load_shots(shots_path, shots_n, seed=shots_seed)
        prompt_builder = with_few_shot(prompt_builder_base, shots=shots, max_k=USE_FIXED_K)
    else:
        print("[Benchmark] zero-shot (no exemplars)")

    # Check whether we're using a fixed K or not
    if USE_FIXED_K is not None:
        print(f"[Track C] Using FIXED K = {USE_FIXED_K} from best_heuristic.json (no grid search).")
    else:
        print(f"[Track C] No fixed K provided -> running grid search K∈[{BEST_HEURISTIC.grid_k_min},{BEST_HEURISTIC.grid_k_max}].")

    model, tokenizer = load_model_and_tokenizer(model_name)

    r_scorer = rouge_scorer.RougeScorer(["rouge1","rouge2","rougeL"], use_stemmer=True)

    P_list, R_list, F_list = [], [], []
    R1_list, R2_list, RL_list = [], [], []

    t0 = time.perf_counter()
    for i, row in df.iterrows():
        sentences = list(row["sentences"])
        ref_abs = row["abstractive_summary"]
        gold_idx0 = list(row["selected_indices"])  # 0-based
        gold_1b = [x + 1 for x in gold_idx0]

        # ensure predict_indices_for_doc can call builder with/without max_k safely
        pred = predict_indices_for_doc(model, tokenizer, prompt_builder, sentences)

        P, R, F = prf1(set(pred), set(gold_1b))
        P_list.append(P); R_list.append(R); F_list.append(F)

        # ROUGE vs abstractive reference
        hyp = indices_to_text(sentences, pred)
        rs = r_scorer.score(ref_abs, hyp)
        R1_list.append(rs["rouge1"].fmeasure)
        R2_list.append(rs["rouge2"].fmeasure)
        RL_list.append(rs["rougeL"].fmeasure)

        if (i+1) % 20 == 0:
            print(f"[{i+1}/{len(df)}] P={P:.3f} R={R:.3f} F1={F:.3f} | "
                  f"R1={R1_list[-1]:.3f} R2={R2_list[-1]:.3f} RL={RL_list[-1]:.3f}")

    dt = time.perf_counter() - t0
    metrics = {
        "precision": float(np.mean(P_list)) if P_list else 0.0,
        "recall":    float(np.mean(R_list)) if R_list else 0.0,
        "f1":        float(np.mean(F_list)) if F_list else 0.0,
        "rouge1":    float(np.mean(R1_list)) if R1_list else 0.0,
        "rouge2":    float(np.mean(R2_list)) if R2_list else 0.0,
        "rougeL":    float(np.mean(RL_list)) if RL_list else 0.0,
        "num_docs": int(len(df)),
        "model": model_name,
        "strategy": strategy,
        "elapsed_sec": dt,
        # provenance for reproducibility
        "few_shot": bool(shots_path and shots_n > 0),
        "shots_path": shots_path,
        "shots_n": int(shots_n),
        "shots_seed": int(shots_seed),
        "use_fixed_k": USE_FIXED_K if USE_FIXED_K is not None else None,
    }

    print("\n=== Benchmark Summary ===")
    print(json.dumps(metrics, indent=2))
    base = os.path.splitext(silver_path)[0]
    out_metrics = f"{base}__bench_{strategy.replace('/','_')}__{model_name.split('/')[-1]}.json"
    with open(out_metrics, "w") as f:
        json.dump(metrics, f, indent=2)
    print(f"Saved metrics -> {out_metrics}")



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


[Track C] Using FIXED K = 7 from best_heuristic.json (no grid search).


In [7]:
def main_cli(argv=None):
    parser = argparse.ArgumentParser(description="Track C: Silver generation + Benchmarking")
    sub = parser.add_subparsers(dest="cmd", required=True)

    # make-silver
    p_make = sub.add_parser("make-silver", help="Generate silver extractive labels from an abstractive dataset")
    p_make.add_argument("--dataset", required=True, choices=["cnndm","xsum"])
    p_make.add_argument("--split", default="test")
    p_make.add_argument("--out", required=True)
    p_make.add_argument("--max-docs", type=int, default=None)
    p_make.add_argument("--heuristic", choices=["global","beam","greedy","local"], default=BEST_HEURISTIC.name)
    p_make.add_argument("--rouge", choices=["rouge1","rouge2","rougeL"], default=BEST_HEURISTIC.eval_criteria)
    p_make.add_argument("--kmin", type=int, default=BEST_HEURISTIC.grid_k_min)
    p_make.add_argument("--kmax", type=int, default=BEST_HEURISTIC.grid_k_max)
    p_make.add_argument("--beam-size", type=int, default=BEST_HEURISTIC.beam_size)

    # benchmark
    p_bench = sub.add_parser("benchmark", help="Run LLM prompting on a silver dataset")
    p_bench.add_argument("--silver", required=True)
    p_bench.add_argument("--model", required=True)
    p_bench.add_argument("--strategy", choices=list(PROMPT_STRATEGIES.keys()), default="vanilla")
    p_bench.add_argument("--max-docs", type=int, default=None)

    # few-shot flags
    p_bench.add_argument("--shots-path", type=str, default=None,
                         help="Path to JSONL exemplars from validation silver.")
    p_bench.add_argument("--shots-n", type=int, default=0,
                         help="Number of exemplars to include (0 = zero-shot).")
    p_bench.add_argument("--shots-seed", type=int, default=SEED,
                     help="RNG seed for shuffling/choosing exemplars in the prompt.")


    # make-shots
    p_shots = sub.add_parser("make-shots", help="Create few-shot exemplars (JSONL) from a silver parquet (validation split)")
    p_shots.add_argument("--silver", required=True)
    p_shots.add_argument("--out", required=True)
    p_shots.add_argument("--k", type=int, default=3)
    p_shots.add_argument("--max-sents", type=int, default=8)
    p_shots.add_argument("--max-chars", type=int, default=220)
    p_shots.add_argument("--seed", type=int, default=SEED, help="RNG seed for sampling exemplars.")

    args = parser.parse_args(argv)

    if args.cmd == "make-silver":
        cfg = HeuristicConfig(
            name=args.heuristic,
            eval_criteria=args.rouge,
            grid_k_min=args.kmin,
            grid_k_max=args.kmax,
            beam_size=args.beam_size
        )
        make_silver(args.dataset, args.split, args.out, args.max_docs, cfg)

    elif args.cmd == "benchmark":
        run_benchmark(
            silver_path=args.silver,
            model_name=args.model,
            strategy=args.strategy,
            max_docs=args.max_docs,
            shots_path=args.shots_path,
            shots_n=args.shots_n,
            shots_seed=args.shots_seed,
            )

    elif args.cmd == "make-shots":
        make_shots_from_silver(
            silver_path=args.silver,
            out_path=args.out,
            k=args.k,
            max_sents=args.max_sents,
            max_chars_per_sent=args.max_chars,
            seed=args.seed,
        )
if __name__ == "__main__":
  import sys, os
  if hasattr(sys, "ps1") or "JPY_PARENT_PID" in os.environ:
      pass
  else:
      main_cli()


**Few shot inference:**

*   Dataset: XSum
*   Model: Qwen/Qwen2.5-3B-Instruct
*   Prompting technique: LTM



In [8]:

try:
    _ = _load_xsum_parquet("test").select(range(3))
    print("[smoke] xsum parquet loader OK")
except Exception as e:
    print("[smoke] xsum parquet loader failed:", e)


# Build SILVER on XSum VALIDATION (for few-shot exemplars)
main_cli([
    "make-silver",
    "--dataset", "xsum",
    "--split", "validation",
    "--out", "out/xsum_silver_val.parquet",
    "--max-docs", "200"
])

# Create few-shot exemplars from the validation silver
main_cli([
    "make-shots",
    "--silver", "out/xsum_silver_val.parquet",
    "--out", "out/xsum_shots.jsonl",
    "--k", "3",               # number of exemplars to include
    "--max-sents", "8",       # truncate per-exemplar doc length to keep prompts short
    "--max-chars", "220"      # clip long sentences
])

# Build SILVER on XSum TEST (for benchmarking)
main_cli([
    "make-silver",
    "--dataset", "xsum",
    "--split", "test",
    "--out", "out/xsum_silver.parquet",
    "--max-docs", "100"
])

# Benchmark with FEW-SHOT
main_cli([
    "benchmark",
    "--silver", "out/xsum_silver.parquet",
    "--model", "Qwen/Qwen2.5-3B-Instruct",
    "--strategy", "ltm",
    "--max-docs", "100",
    "--shots-path", "out/xsum_shots.jsonl",
    "--shots-n", "3"
])


default/test/0000.parquet:   0%|          | 0.00/17.0M [00:00<?, ?B/s]

Generating test split: 0 examples [00:00, ? examples/s]

[smoke] xsum parquet loader OK


default/validation/0000.parquet:   0%|          | 0.00/16.7M [00:00<?, ?B/s]

Generating validation split: 0 examples [00:00, ? examples/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


[20/100] P=0.714 R=0.714 F1=0.714 | R1=0.102 R2=0.028 RL=0.074
[40/100] P=0.286 R=0.286 F1=0.286 | R1=0.088 R2=0.006 RL=0.057
[60/100] P=0.857 R=0.857 F1=0.857 | R1=0.192 R2=0.062 RL=0.096
[80/100] P=0.571 R=0.571 F1=0.571 | R1=0.091 R2=0.013 RL=0.078
[100/100] P=1.000 R=1.000 F1=1.000 | R1=0.100 R2=0.033 RL=0.075

=== Benchmark Summary ===
{
  "precision": 0.6888095238095236,
  "recall": 0.6766904761904762,
  "f1": 0.6821556221556222,
  "rouge1": 0.12852152318460125,
  "rouge2": 0.02776734687220196,
  "rougeL": 0.08520878598182512,
  "num_docs": 100,
  "model": "Qwen/Qwen2.5-3B-Instruct",
  "strategy": "ltm",
  "elapsed_sec": 104.44722130900004,
  "few_shot": true,
  "shots_path": "out/xsum_shots.jsonl",
  "shots_n": 3,
  "shots_seed": 42,
  "use_fixed_k": 7
}
Saved metrics -> out/xsum_silver__bench_ltm__Qwen2.5-3B-Instruct.json


**Zero shot inference:**

*   Dataset: XSum
*   Model: Qwen/Qwen2.5-3B-Instruct
*   Prompting technique: LTM


In [9]:
try:
    _ = _load_xsum_parquet("test").select(range(3))
    print("[smoke] xsum parquet loader OK")
except Exception as e:
    print("[smoke] xsum parquet loader failed:", e)


# build silver on XSum
main_cli([
    "make-silver",
    "--dataset", "xsum",
    "--split", "test",
    "--out", "out/xsum_silver.parquet",
    "--max-docs", "100"
])

# benchmark
main_cli([
    "benchmark",
    "--silver", "out/xsum_silver.parquet",
    "--model", "Qwen/Qwen2.5-3B-Instruct",
    "--strategy", "ltm",
    "--max-docs", "100"
])




[smoke] xsum parquet loader OK
[xsum] processed 50/100
[xsum] processed 100/100
Finished xsum:test -> 100 examples in 60.6s
Saved silver dataset -> out/xsum_silver.parquet
[Benchmark] zero-shot (no exemplars)
[Track C] Using FIXED K = 7 from best_heuristic.json (no grid search).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[20/100] P=0.571 R=0.571 F1=0.571 | R1=0.089 R2=0.027 RL=0.071
[40/100] P=0.143 R=0.143 F1=0.143 | R1=0.077 R2=0.006 RL=0.058
[60/100] P=0.857 R=0.857 F1=0.857 | R1=0.184 R2=0.059 RL=0.109
[80/100] P=0.571 R=0.571 F1=0.571 | R1=0.090 R2=0.013 RL=0.077
[100/100] P=1.000 R=1.000 F1=1.000 | R1=0.100 R2=0.033 RL=0.075

=== Benchmark Summary ===
{
  "precision": 0.6880476190476189,
  "recall": 0.6679285714285713,
  "f1": 0.6758088578088577,
  "rouge1": 0.12865012575171425,
  "rouge2": 0.02810122367502255,
  "rougeL": 0.08470344128132622,
  "num_docs": 100,
  "model": "Qwen/Qwen2.5-3B-Instruct",
  "strategy": "ltm",
  "elapsed_sec": 109.23480051699994,
  "few_shot": false,
  "shots_path": null,
  "shots_n": 0,
  "shots_seed": 42,
  "use_fixed_k": 7
}
Saved metrics -> out/xsum_silver__bench_ltm__Qwen2.5-3B-Instruct.json


**Zero shot inference:**

*   Dataset: CNN/DM
*   Model: Qwen/Qwen2.5-3B-Instruct
*   Prompting technique: LTM


In [10]:

_ = load_dataset("cnn_dailymail", "3.0.0", split="test[:3]")

# build silver on CNN/DM
main_cli([
    "make-silver",
    "--dataset","cnndm",
    "--split","test",
    "--out","out/cnndm_silver.parquet",
    "--max-docs","100"
])

# benchmark
main_cli([
    "benchmark",
    "--silver","out/cnndm_silver.parquet",
    "--model","Qwen/Qwen2.5-3B-Instruct", #"Qwen/Qwen2.5-3B-Instruct", "meta-llama/Llama-3.1-8B-Instruct"
    "--strategy","ltm",
    "--max-docs","100"
])



README.md: 0.00B [00:00, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

...

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

[cnndm] processed 50/100
[cnndm] processed 100/100
Finished cnndm:test -> 100 examples in 93.0s
Saved silver dataset -> out/cnndm_silver.parquet
[Benchmark] zero-shot (no exemplars)
[Track C] Using FIXED K = 7 from best_heuristic.json (no grid search).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

[20/100] P=0.833 R=0.714 F1=0.769 | R1=0.179 R2=0.067 RL=0.100
[40/100] P=0.143 R=0.143 F1=0.143 | R1=0.286 R2=0.157 RL=0.286
[60/100] P=0.714 R=0.714 F1=0.714 | R1=0.144 R2=0.088 RL=0.106
[80/100] P=1.000 R=1.000 F1=1.000 | R1=0.213 R2=0.108 RL=0.178
[100/100] P=1.000 R=1.000 F1=1.000 | R1=0.175 R2=0.061 RL=0.121

=== Benchmark Summary ===
{
  "precision": 0.5635714285714285,
  "recall": 0.5566666666666668,
  "f1": 0.5598301698301698,
  "rouge1": 0.21832519933522263,
  "rouge2": 0.0962549752559967,
  "rougeL": 0.15536125452155974,
  "num_docs": 100,
  "model": "Qwen/Qwen2.5-3B-Instruct",
  "strategy": "ltm",
  "elapsed_sec": 118.60295888499991,
  "few_shot": false,
  "shots_path": null,
  "shots_n": 0,
  "shots_seed": 42,
  "use_fixed_k": 7
}
Saved metrics -> out/cnndm_silver__bench_ltm__Qwen2.5-3B-Instruct.json


In [ ]:
from pathlib import Path
import json

# check file exists
print(Path("best_heuristic.json").resolve())

# load
with open("best_heuristic.json", "r") as f:
    best_cfg = json.load(f)

print(best_cfg)
